In [1]:
!pip install wandb
import wandb
# Replace with your actual API key
api_key = "8f58df9a66485e9ea9149b8b599cb14eb71832dc"

# Login to Weights & Biases
wandb.login(key=api_key)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.2 MB/s eta 0:00:00


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [2]:
import gym
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (16, 10)
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
torch.manual_seed(0)
import base64
import io  # For visualization
from gym.wrappers.monitoring import video_recorder
from IPython.display import HTML
from IPython import display
import glob


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

env = gym.make('CartPole-v1')
env.seed(0)
print('observation space:', env.observation_space)
print('action space:', env.action_space)

wandb.init(project="R_CartPole_baseline", config={  # Initialize wandb first
    "hidden_size": 32,  # Default values
    "n_episodes": 1000,
    "max_t": 1000,
    "gamma": 0.99,
    "print_every": 100,
    "lr": 1e-2,
})

config = {
    "hidden_size": wandb.config.hidden_size,  # Access wandb.config after initialization
    "n_episodes": wandb.config.n_episodes,
    "max_t": wandb.config.max_t,
    "gamma": wandb.config.gamma,
    "print_every": wandb.config.print_every,
    "lr": wandb.config.lr,
}

class Policy(nn.Module):
    def __init__(self, state_size=4, action_size=2, hidden_size=config["hidden_size"]):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, action_size)

    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = self.fc2(x)
        # we just consider 1 dimensional probability of action
        return F.softmax(x, dim=1)

    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        if state.shape[1] != 4:  # Check if the state tensor has the expected shape
            state = state.view(1, 4)  # Reshape the state tensor to (1, 4)
        probs = self.forward(state).cpu()
        model = Categorical(probs)
        action = model.sample()
        return action.item(), model.log_prob(action)


class Baseline(nn.Module):
    def __init__(self, state_size, hidden_size=32):
        super(Baseline, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)

    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = self.fc2(x)
        return x



def reinforce(env, policy, optimizer, baseline, baseline_optimizer, n_episodes=1000, max_t=1000, gamma=1.0, print_every=100):
    scores_deque = deque(maxlen=100)
    scores = []
    for e in range(1, n_episodes):
        saved_log_probs = []
        rewards = []
        states = []
        state = env.reset()
        for t in range(max_t):
            states.append(state)
            action, log_prob = policy.act(state)
            saved_log_probs.append(log_prob)
            state, reward, done, _ = env.step(action)
            rewards.append(reward)
            if done:
                break
        scores_deque.append(sum(rewards))
        scores.append(sum(rewards))
        discounts = [gamma ** i for i in range(len(rewards) + 1)]
        R = sum([a * b for a, b in zip(discounts, rewards)])

        # Calculate the baseline
        baseline_values = []
        for state_in_trajectory in states:
            state_tensor = torch.from_numpy(state_in_trajectory).float().unsqueeze(0).to(device)
            baseline_value = baseline(state_tensor)
            baseline_values.append(baseline_value)
        baseline_values = torch.cat(baseline_values, dim=0).requires_grad_()

        # Calculate the policy loss
        policy_loss = []
        for log_prob, baseline_value in zip(saved_log_probs, baseline_values):
            policy_loss.append(-(log_prob * (R - baseline_value)))
        policy_loss = torch.cat(policy_loss).sum()

        # Update the policy
        optimizer.zero_grad()
        policy_loss.backward(retain_graph=True)  # Retain the computation graph
        optimizer.step()

        # Update the baseline
        baseline_optimizer.zero_grad()
        baseline_loss = ((baseline_values - R) ** 2).mean()
        baseline_loss.backward()
        baseline_optimizer.step()

        if e % print_every == 0:
            print('Episode {}\\tAverage Score: {:.2f}'.format(e, np.mean(scores_deque)))
        if np.mean(scores_deque) >= 195.0:
            print(f'Environment {env.unwrapped.spec.id} solved in {e - 100:d} episodes!\\tAverage Score: {np.mean(scores_deque):.2f}')
            break

    return scores

############################################For Regret minimization###################################

# def reinforce(policy, optimizer, n_episodes=config["n_episodes"], max_t=config["max_t"], gamma=config["gamma"], print_every=config["print_every"]):
#     scores_deque = deque(maxlen=100)
#     scores = []
#     for e in range(1, n_episodes):
#         # ... (the rest of the code remains the same)
#         if e % print_every == 0:
#             print('Episode {}\\tAverage Score: {:.2f}'.format(e, np.mean(scores_deque)))
#             regret = 500 - np.mean(scores_deque)  # Calculate regret
#             wandb.log({"Regret": regret})
#         if np.mean(scores_deque) >= 495.0:  # Adjust the threshold for early stopping
#             print('Environment solved in {:d} episodes!\\tAverage Score: {:.2f}'.format(e - 100, np.mean(scores_deque)))
#             break
#     wandb.run.summary["final_regret"] = 500 - np.mean(scores_deque)  # Log final regret
#     return scores




#############################################################################

wandb.init(project="R_CartPole_baseline", config=config)  # Initialize wandb

policy = Policy().to(device)
optimizer = optim.Adam(policy.parameters(), lr=1e-2)
baseline = Baseline(state_size=env.observation_space.shape[0]).to(device)
baseline_optimizer = optim.Adam(baseline.parameters(), lr=1e-2)
scores = reinforce(env, policy, optimizer, baseline, baseline_optimizer, n_episodes=2000)
wandb.finish()  # Finish wandb run

cpu
observation space: Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
action space: Discrete(2)


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:256: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` instead.
  deprecation(
wandb: Currently logged in as: bhavik-160990105023. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode 100\tAverage Score: 29.38
Episode 200\tAverage Score: 64.28
Episode 300\tAverage Score: 115.99
Environment CartPole-v1 solved in 246 episodes!\tAverage Score: 195.18


In [3]:
# print(env.spec.reward_threshold)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
sweep_config = {
    'method': 'grid',
    'metric': {
      'name': 'Average Reward',
      'goal': 'maximize'
    },
    'parameters': {
        'hidden_size': {
            'values': [32,64,128]
        },
        'max_t': {
            'values': [500, 1000, 1500]
        },
        'lr': {
            'values': [1e-5, 1e-3,1e-4]
        }
    }
}

In [5]:
# Initialize a new sweep
sweep_id = wandb.sweep(sweep_config, entity="bhavik-160990105023", project="R_CartPole_baseline")

Create sweep with ID: thm46x8i
Sweep URL: https://wandb.ai/bhavik-160990105023/R_CartPole_baseline/sweeps/thm46x8i


In [6]:
def train(config=None):
    # Set default values for hyperparameters
    default_config = {
        "hidden_size": 32,
        "n_episodes": 1000,
        "max_t": 1000,
        "gamma": 1.0,
        "print_every": 100,
        "lr": 1e-2,
        "baseline_lr": 1e-2,
    }

    # Initialize a new wandb run
    run = wandb.init(config=config, reinit=True)

    # If config is None, use the default values
    if config is None:
        run.config.update(default_config, allow_val_change=True)
    config = run.config

    # Initialize the environment and seed
    env = gym.make('CartPole-v1')
    env.seed(0)

    # Create the policy network with the specified hyperparameters
    policy = Policy(state_size=4, action_size=2, hidden_size=config.hidden_size).to(device)
    optimizer = optim.Adam(policy.parameters(), lr=config.lr)

    # Create the baseline network with the specified hyperparameters
    baseline = Baseline(state_size=4).to(device)
    baseline_optimizer = optim.Adam(baseline.parameters(), lr=config.baseline_lr)

    # Run the REINFORCE algorithm with the specified hyperparameters
    scores = reinforce(env, policy, optimizer, baseline, baseline_optimizer, n_episodes=config.n_episodes, max_t=config.max_t, gamma=config.gamma, print_every=config.print_every)

    # Log the final score as a summary metric
    run.summary["final_score"] = np.mean(scores[-100:])

    # Finish the wandb run
    run.finish()

    # Return any necessary values or metrics
    return np.mean(scores[-100:])

In [7]:
sweep_id = wandb.sweep(sweep_config, project="R_CartPole_baseline")
wandb.agent(sweep_id, train, count=50)

Create sweep with ID: 5w8xruix
Sweep URL: https://wandb.ai/bhavik-160990105023/R_CartPole_baseline/sweeps/5w8xruix


wandb: Agent Starting Run: 9fs04x5x with config:
wandb: 	hidden_size: 32
wandb: 	lr: 1e-05
wandb: 	max_t: 500


wandb: WARNING Config item 'hidden_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'max_t' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'lr' was locked by 'sweep' (ignored update).
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:256: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Pl

Episode 100\tAverage Score: 18.98
Episode 200\tAverage Score: 17.61
Episode 300\tAverage Score: 19.51
Episode 400\tAverage Score: 17.40
Episode 500\tAverage Score: 16.19
Episode 600\tAverage Score: 18.95
Episode 700\tAverage Score: 18.32
Episode 800\tAverage Score: 19.38
Episode 900\tAverage Score: 16.85


final_score,18.63


...

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 1kz8moh0 with config:
wandb: 	hidden_size: 128
wandb: 	lr: 0.0001
wandb: 	max_t: 1500


wandb: WARNING Config item 'hidden_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'max_t' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'lr' was locked by 'sweep' (ignored update).


Episode 100\tAverage Score: 22.14
Episode 200\tAverage Score: 23.99
Episode 300\tAverage Score: 22.11
Episode 400\tAverage Score: 25.66
Episode 500\tAverage Score: 27.67
Episode 600\tAverage Score: 29.48
Episode 700\tAverage Score: 37.31
Episode 800\tAverage Score: 33.07
Episode 900\tAverage Score: 39.55


final_score,40.45


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [8]:
# wandb agent <sweep_id>